# Dados de Análise de Credito

Machine learning e data science são ótimas ferramentas no mercado financeiro. Tais tecnlogias permitem realizar análises e auxiliar na tomada de decisões da empresa, levando em consideração o histórico de dados reservados.
O intuito dessa base de dados é analisar a possível base de créditos a novos clientes e compreender o procedimento de pré-processamento de dados e estudos com a biblioteca Scikit-learn e o algoritmo de Naive Bayes.

## Análise de Dados

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#base de dados
base = pd.read_csv("credit_data.csv")
base

,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1
...,...,...,...,...,...
1995,1996,59221.044874,48.518179,1926.729397,0
1996,1997,69516.127573,23.162104,3503.176156,0
1997,1998,44311.449262,28.017167,5522.786693,1
1998,1999,43756.056605,63.971796,1622.722598,0


Em uma breve de análise de dados, conseguimos perceber inicialmente que temos os clientes identificados pelos seus id's. 

Os dados revelam que o cliente id consiste em uma categoria de variáveis nominal. Conseguimos perceber também novos tipos de variaveis como, na coluna 2 (income), que consistem de dados de variáveis numéricas contínuas e a idade (age), no formato apresentado pelos dados, consistem também de variáveis numéricas contínuas. a coluna Loan - que é a coluna de valor de crédito - consistem em dados continuos. 

Por último, temos a saída de dados (default) sendo 0 e 1, variáveis discretas.

In [3]:
#descrição dos dados
base.describe()

,i#clientid,income,age,loan,c#default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.807559,4444.369695,0.141500
std,577.494589,14326.327119,13.624469,3045.410024,0.348624
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,500.750000,32796.459717,28.990415,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


## Tratamento de dados e pré processamento

Nessa etapa será de extrema importância tratar os dados de créditos irregulares. na tabela acima, temos na coluna age (idade), um min de -52,423280, ou seja, uma idade negativa. Isso consiste de erro dos dados. Tais erros podem atrapalhar em nossos modelos de machine learning para predições.

In [4]:
#Localizando os dados da coluna de idade com valores negativos
base.loc[base['age'] <0]

,i#clientid,income,age,loan,c#default
15,16,50501.726689,-28.218361,3977.287432,0
21,22,32197.620701,-52.423280,4244.057136,0
26,27,63287.038908,-36.496976,9595.286289,0


O método utilizado para preencher esses dados de idades negativas é a substituição da média das idades (média sem esses dados negativos) no valores negativos.

In [5]:
#preenchendo os valores dos dados negativos com a média aritmética das idades

#média das pessoas com idade > 0
base.mean()
media_idade_positiva = base['age'][base.age > 0].mean()
print(f'A media dos dados de idade com os valores positivos é {media_idade_positiva} \n\n')

base.loc[base.age < 0, 'age'] = 40.92

A media dos dados de idade com os valores positivos é 40.92770044906149 




In [7]:
#Dividindo os dados entre previsores e classe
previsores = base.iloc[:, 1:4].values
classe = base.iloc[:, 4].values

In [8]:
#Tratando valores NaN com a media
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(previsores[:, 1:4])
previsores[:, 1:4] = imputer.transform(previsores[:, 1:4])

In [9]:
#Transformando todos os atributos numéricos na mesma escala
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

In [11]:
#Dividindo a base de dados entre treinamento e teste
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.25, random_state=0)

## Treino e teste dos dados com Machine Learning: utilização do algoritmo de Naive Bayes

In [12]:
#treinando o modelo
from sklearn.naive_bayes import GaussianNB

classificador = GaussianNB()
classificador.fit(previsores_treinamento,classe_treinamento)

GaussianNB(priors=None, var_smoothing=1e-09)

In [13]:
#Realizando a previsão
previsoes = classificador.predict(previsores_teste)
previsoes

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [17]:
#Comparando os dados de classes e verificando a precisão do algoritmo
from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste,previsoes)
precisao_porcentagem = precisao * 100
print(f'A Precisão do algoritmo criado é de: {precisao_porcentagem}%')

A Precisão do algoritmo criado é de: 93.8%


Comparando com a classe de teste o algoritmo apresentou uma acuracia de 93,8%. Isso representa uma alto índice de acerto do algoritmo, o tornando, interessante para a tomada de decisões.

In [16]:
matriz = confusion_matrix(classe_teste,previsoes)
matriz

array([[428,   8],
       [ 23,  41]])

Pela matriz de confusão podemos perceber que dos 500 dados previsores o algoritmo conseguiu acertar 428 verdadeiros positivos e 41 verdadeiros negativos. acertando um total de 466 dados analisados corretamentecontendo apenas 34 erros de um total de 500 dados de análise de crédito

## Conclusão

A utilização de machine learning consiste em uma aplicação prática do uso de estatísticas, utilizando ferramentas de tecnologia, como a linguagem Python. O intuito desse estudo é apresentar o algorítmo de Naive Bayes de forma prática, simples e rápida. Nesse contexto, o algoritmo se mostrou eficaz trazendo uma acurácia de 93,8%. Importante destacar também que o trabalho de um data scientist não termina aqui: é importante testar novos algoritmos e refazer as análises de pré-processamento, limpeza dos dados.